## Using Word2Vec and Doc2Vec to get numeric representations of YouTube video
## comments

In [1]:
!pip install -U gensim
from gensim.models import doc2vec, Doc2Vec

Requirement already up-to-date: gensim in c:\programdata\anaconda3\lib\site-packages (3.7.3)


In [2]:
from collections import OrderedDict
import pandas as pd
import re

In [4]:
videos_df = pd.read_csv('new_videos.csv')  # note that this csv data does not have comments column
videos_df.head()

,video_title,video_id,moods
0,"[FREE DL] Dave East Type Beat ""Momma Workin"" (...",kQKLl6jXipQ,CHILL
1,Logic - Indica Badu ft. Wiz Khalifa [Instrumen...,-QQUaWtMW3w,HAPPY
2,【Off Vocal】Yasuha. - I Lost Your Love feat. Ha...,1QejWtVKE8s,CHILL
3,Shiva In Exile - Earth Tone (Instrumental),4zrLieLZ_i8,CHILL
4,8 Remix - Almighty ft. Varios [INSTRUMENTAL + ...,kmLvR68_nLg,CONFIDENT-SASSY


In [27]:
comments_df = pd.read_csv('new_comments.csv', encoding="utf-8")  # note no moods column
comments_df.head()

,video_id,comments
0,2raUmsKIG6U,"['That is some cool grafiti song, sick!', 'Dam..."
1,3XAkH0w3QLc,"['the ""jake young"" at the begining scared me l..."
2,hXe0Elz7WV8,['This is good. Definitely on favorite song li...
3,x1frIzjJv5E,['I think I would sound better at the original...
4,iPqaRRdb7eQ,[]


In [7]:
# full_videos_df = pd.merge(videos_df, comments_df, on='video_id')
# use above line as example, if need to merge new_comments and new_videos csv's

In [ ]:
from nltk.tokenize import sent_tokenize

### Preprocessing and tokenizing the comments data

In [9]:
# Examine the raw comments for the first video
first_vid_comments = comments_df.comments.values[0]
first_vid_comments  # NOTE: this whole object is a string; change it to a list with strings inside

'[\'That is some cool grafiti song, sick!\', \'Damn this is dope, I miss the 90s\', \'Nice beat and cuts man, unfortunately they faded so early.\', \'Very nice beat bra\', \'Can you feel IT can you feel IT can you feeeeel itttt\', \'this is super golden age hip hop, but still so atmospheric with that string sample!! i just dropped a new vid on my channel would love to hear the opinion of a true artist like you over there! love this beat tho bro!\', \'Gj /GoodJob!\', \'Se puede sacar una super improvisacion buen beat\', "Hello. I want to use ur music for a dance project but i can\'t find you on facebook, and also not on soundcloud. How can i get your music? Thank you", "who\'s down to start a rap group", \'You bouta get the cred for some slick shit bro this beat is fucking absurd\', "It\'s ok if not", \'Can I use this brother?\', \'Hola man queria saber si podia utilizar este  beat para un tema que tengo pensado (se llama "mientras suenan las sirenas ) y es justo lo que me imagine para 

In [ ]:
# Tried various operations on first video comments
# Create a list object of first video comments by splitting on "\', " -- not successful

In [10]:
# Let's see the raw comments for the second video
second_vid_comments = comments_df.comments.values[1]
second_vid_comments

'[\'the "jake young" at the begining scared me lolol\', \'oml there are literally 4-5 comments with lyrics i cant-\', \'NNA NANANA NANAN NANANNAANNNAANANANANANANANANANANANNNNANANANANANANANANAN*YUMINOSHIi?b0AI\', \'Chi è qui per Klaus? 🇮🇹🇮🇹🇮🇹\', \'great job, can I use it for a cover? :) thank you\', \'Shut your mouth, I don’t wanna fuckin breath now\\nAll the people on my back I’m getting weighed down \\nMy future’s falling to the floor, I’m getting worried now\\nWill I make it in this game or will I fuckin drown\\n\\nI can’t fuckin sleep\\nI can’t fuckin breath \\nAll these pills are on my mind, now I’m in this deep\\nFinally passed out on the floor from all this codeine \\nShe be screaming I want more until she fuck me\\n\\nLife’s a bitter mess when you are depressed \\nAll these demons in my head I’m feeling possessed\\nI got problems with these women, yeah I’m obsessed \\nEvery day I’m working hard creating progress \\n\\nWhen I wake up and I know you’re on my fucking mind \\nI can’

In [ ]:
# Study all comments for all videos
# Based on methods from aws-sagemaker-python-notebook [not yet updated on github]
all_comments = []
all_comments_dict = {}
for comment_group in list(comments_df.comments):
    comment_group = comment_group.replace('[', '')
    comment_group = comment_group.replace(']', '')
    comment_group = comment_group.replace("'", '')
    all_comments.append(comment_group)
    # if ', ' in comment_group:
    #     comment_group = comment_group.split(', ')
    #     for item in comment_group:
    #         all_comments.append(comment_group)
    # print(comment_group, len(comment_group))

print('length of all_comments is:', len(all_comments))
print(all_comments[:2])

In [28]:
# Convert text to lower-case, do several char replacements, and create list by splitting comments string

# Create a function to do this when applied to all values in df's comments column

def normalize_text(comment_group):
    norm_comment_group = comment_group.lower()
    norm_comment_group = re.sub(r"([\.\",\(\)!\?;:])", " \\1 ", norm_comment_group)
    norm_comment_group = norm_comment_group.replace('\\n', ', ')
    norm_comment_group = norm_comment_group.replace('"', "\'")
    while "\'" in norm_comment_group:
        norm_comment_group = norm_comment_group.replace("\'", '')
    norm_comment_group = norm_comment_group.replace('[', '')
    norm_comment_group = norm_comment_group.replace(']', '')
    return norm_comment_group

# Apply normalize_text function to all comments column values

comments_df.comments = comments_df.comments.apply(lambda x: normalize_text(x))

In [29]:
comments_df.comments.values[:2]

array(['that is some cool grafiti song ,  sick !  ,  damn this is dope ,  i miss the 90s ,  nice beat and cuts man ,  unfortunately they faded so early .  ,  very nice beat bra ,  can you feel it can you feel it can you feeeeel itttt ,  this is super golden age hip hop ,  but still so atmospheric with that string sample !  !  i just dropped a new vid on my channel would love to hear the opinion of a true artist like you over there !  love this beat tho bro !  ,  gj /goodjob !  ,  se puede sacar una super improvisacion buen beat ,    hello .  i want to use ur music for a dance project but i cant find you on facebook ,  and also not on soundcloud .  how can i get your music ?  thank you   ,    whos down to start a rap group   ,  you bouta get the cred for some slick shit bro this beat is fucking absurd ,    its ok if not   ,  can i use this brother ?  ,  hola man queria saber si podia utilizar este  beat para un tema que tengo pensado  ( se llama   mientras suenan las sirenas  )  y es ju

In [30]:
# The fourth video, with id iPqaRRdb7eQ, has no comments. 
comments_df.comments.values[4]

''

In [ ]:
sent_tokenize(sent_1)  # Example of using NLTK to tokenize one video's comments as one sentence

### Trying TextBlob for NLP

In [33]:
!pip install textblob
from textblob import TextBlob

  Using cached https://files.pythonhosted.org/packages/60/f0/1d9bfcc8ee6b83472ec571406bd0dd51c0e6330ff1a51b2d29861d389e85/textblob-0.15.3-py2.py3-none-any.whl


In [34]:
# Let's work with the first video's comments, to start

blob = TextBlob(comments_df.comments.values[0])

In [35]:
# See if we can get tags for the first video's comments

blob.tags

[('that', 'DT'),
 ('is', 'VBZ'),
 ('some', 'DT'),
 ('cool', 'JJ'),
 ('grafiti', 'NN'),
 ('song', 'NN'),
 ('sick', 'NN'),
 ('damn', 'NN'),
 ('this', 'DT'),
 ('is', 'VBZ'),
 ('dope', 'NN'),
 ('i', 'JJ'),
 ('miss', 'VBP'),
 ('the', 'DT'),
 ('90s', 'CD'),
 ('nice', 'JJ'),
 ('beat', 'NN'),
 ('and', 'CC'),
 ('cuts', 'NNS'),
 ('man', 'NN'),
 ('unfortunately', 'RB'),
 ('they', 'PRP'),
 ('faded', 'VBD'),
 ('so', 'RB'),
 ('early', 'JJ'),
 ('very', 'RB'),
 ('nice', 'JJ'),
 ('beat', 'NN'),
 ('bra', 'NN'),
 ('can', 'MD'),
 ('you', 'PRP'),
 ('feel', 'VB'),
 ('it', 'PRP'),
 ('can', 'MD'),
 ('you', 'PRP'),
 ('feel', 'VB'),
 ('it', 'PRP'),
 ('can', 'MD'),
 ('you', 'PRP'),
 ('feeeeel', 'VB'),
 ('itttt', 'RB'),
 ('this', 'DT'),
 ('is', 'VBZ'),
 ('super', 'JJ'),
 ('golden', 'JJ'),
 ('age', 'NN'),
 ('hip', 'NN'),
 ('hop', 'NN'),
 ('but', 'CC'),
 ('still', 'RB'),
 ('so', 'RB'),
 ('atmospheric', 'JJ'),
 ('with', 'IN'),
 ('that', 'DT'),
 ('string', 'VBG'),
 ('sample', 'NN'),
 ('i', 'NN'),
 ('just', 'RB'),
 ('

In [55]:
from statistics import mean

polarity_list = []
subjectivity_list = []
for sentence in blob.sentences:
    print('polarity and subjectivity for this sentence are:', sentence.sentiment)
    polarity = sentence.sentiment.polarity
    subjectivity = sentence.sentiment.subjectivity
    if polarity !=0 :
        polarity_list.append(polarity)
    if subjectivity !=0 :
        subjectivity_list.append(subjectivity)
print('polarity_list is:', polarity_list, '\n', 'subjectivity_list is:', subjectivity_list)

polarity and subjectivity for this sentence are: Sentiment(polarity=-0.27142857142857146, subjectivity=0.7535714285714286)
polarity and subjectivity for this sentence are: Sentiment(polarity=0.06666666666666667, subjectivity=0.7666666666666666)
polarity and subjectivity for this sentence are: Sentiment(polarity=0.35333333333333333, subjectivity=0.5416666666666666)
polarity and subjectivity for this sentence are: Sentiment(polarity=0.0, subjectivity=0.0)
polarity and subjectivity for this sentence are: Sentiment(polarity=0.3579545454545454, subjectivity=0.5681818181818182)
polarity and subjectivity for this sentence are: Sentiment(polarity=0.625, subjectivity=0.6)
polarity and subjectivity for this sentence are: Sentiment(polarity=0.0, subjectivity=0.0)
polarity and subjectivity for this sentence are: Sentiment(polarity=0.3333333333333333, subjectivity=0.6666666666666666)
polarity and subjectivity for this sentence are: Sentiment(polarity=0.0, subjectivity=0.0)
polarity and subjectivity

In [81]:
# Add two new columns to comments_df: one each for TextBlob polarity mean and subjectivity mean

tblob_vids_comments_df = pd.concat(
    [comments_df, pd.DataFrame(columns=['polarity_mean', 'subjectivity_mean', 'tags'])], axis=1)

# full_videos_df = pd.merge(videos_df, comments_df, on='video_id')
tblob_vids_comments_df = pd.merge(tblob_vids_comments_df, videos_df, on='video_id')
cols = tblob_vids_comments_df.columns.to_list()
cols = [cols[0]] + [cols[5]] + [cols[1]] + cols[2:5] + [cols[-1]]
tblob_vids_comments_df = tblob_vids_comments_df[cols]

In [82]:
tblob_vids_comments_df.head()

,video_id,video_title,comments,polarity_mean,subjectivity_mean,tags,moods
0,2raUmsKIG6U,"[SOLD]"" Graffiti "" - 90s Old School Storytelli...","that is some cool grafiti song , sick ! , d...",NaN,NaN,NaN,CHILL
1,3XAkH0w3QLc,Youth by Daughter | Instrumental (w/ Sheet Music),the jake young at the begining scared me l...,NaN,NaN,NaN,CHILL
2,hXe0Elz7WV8,THE CREATOR LIED TO US (Fansong Instrumental) ...,this is good . definitely on favorite song li...,NaN,NaN,NaN,CONFIDENT-SASSY
3,x1frIzjJv5E,Kendrick Lamar - Blood. (RoadsArt Instrumental...,i think i would sound better at the original s...,NaN,NaN,NaN,CHILL
4,iPqaRRdb7eQ,Nco Qub Hluas Nraug Instrumental + Lyrics - Gi...,,NaN,NaN,NaN,CHILL


In [83]:
# Add TextBlob mean polarity, mean subjectivity, and tags for all comments for each video
# Write functions to do each of the following:
# 1) run TextBlob on comments; 2) populate polarity_mean, subjectivity_mean, and tags values

def get_blob(comments):
    """Use with one video's comments at a time."""
    blob = TextBlob(comments)
    polarity_list = []
    subjectivity_list = []
    for sentence in blob.sentences:
        polarity = sentence.sentiment.polarity
        subjectivity = sentence.sentiment.subjectivity
        if polarity !=0 :
            polarity_list.append(polarity)
        if subjectivity !=0 :
            subjectivity_list.append(subjectivity)
    if len(polarity_list) == 0:
        polarity_list = [0]
    if len(subjectivity_list) == 0:
        subjectivity_list = [0]
    return mean(polarity_list), mean(subjectivity_list), blob.tags


def populate_blob_columns(comms):
    """Use to fill the polarity_mean, subjectivity_mean, and tags columns."""
    polarity_mean, subjectivity_mean, tags = get_blob(comms)
    tblob_vids_comments_df.loc[(tblob_vids_comments_df.comments == comms), 'polarity_mean'] = polarity_mean
    tblob_vids_comments_df.loc[(tblob_vids_comments_df.comments == comms), 'subjectivity_mean'] = subjectivity_mean
    tblob_vids_comments_df.loc[(tblob_vids_comments_df.comments == comms), 'tags'] = [tags]


for comments in tblob_vids_comments_df.comments.values:
    populate_blob_columns(comments)

In [87]:
# Check how df looks, prior to sending to csv
print(tblob_vids_comments_df.shape)
tblob_vids_comments_df.tail(20)

(92, 7)


,video_id,video_title,comments,polarity_mean,subjectivity_mean,tags,moods
72,fWOMRvGMO3s,DAGames - Left Behind [Instrumental] [Remake ...,"no offense but this isn’t accurate . , nice ...",0.66125,0.800833,"[(no, DT), (offense, NN), (but, CC), (this, DT...",ANGRY
73,NNexDl8QF6U,Chidinma - Gone forever | Karaoke Version ( In...,"please do a karaoke of let go locko , th...",0.45,0.658333,"[(please, VB), (do, VB), (a, DT), (karaoke, NN...",SAD
74,eRt8RY71hiY,Teppen E Dash Full Instrumental,"rising ! ! 😍 , fire lemonade rising , inaz...",0.325,0.26,"[(rising, VBG), (😍, NN), (fire, NN), (lemonade...",CHILL
75,FWOjSjqBUuk,Deuce - Nightmare Instrumental (Studio Quality),it would be kickass if someone can take an aca...,0.434331,0.729861,"[(it, PRP), (would, MD), (be, VB), (kickass, V...",ANGRY
76,pFPUeoU418k,"""Dios esta aqui"" Musica Instrumental para orar...","queridos amigos , quiero invitarlos a escucha...",0.125,0.425,"[(queridos, NN), (amigos, NNS), (quiero, NN), ...",SAD
77,bUMFOIAbH4s,♫ Instrumental : Yandere Chan VS Monika - [ EP...,"2 : 40 la partie tros bien , monika extended ...",0.4125,0.7425,"[(2, CD), (40, CD), (la, NN), (partie, NN), (t...",ANGRY
78,WgYXtbtCTzY,Hluas Nraug Vib Nais Instrumental + Lyrics - L...,ua tsaug rau koj cov instrumental os,0,0,"[(ua, JJ), (tsaug, NN), (rau, NN), (koj, NN), ...",CHILL
79,_ntkv1ovpW8,Ib Sim Neej Instrumental + Lyrics - Hmong sad ...,ua neeg byob hauv ntiaj teb nos khos siab heev os,0,0,"[(ua, JJ), (neeg, JJ), (byob, NN), (hauv, NN),...",IN-LOVE
80,eVTfwFk70wc,Dilated Peoples - Basics (Instrumental),one day ill still find this sample .,-0.5,1,"[(one, CD), (day, NN), (ill, VB), (still, RB),...",CONFIDENT-SASSY
81,14Cgt0oQ8aY,"[FREE] Sad Drake x Logic Type Beat ""Haiku"" | C...","drop a comment and i’ll sub to your channel , ...",0.765278,0.683333,"[(drop, NN), (a, DT), (comment, NN), (and, CC)...",CHILL


In [88]:
# Create csv for sharing data outside of notebook

tblob_vids_comments_df.to_csv('videos_with_nlp.csv')

### Creating a Doc2Vec NLP model

In [ ]:
# make a Doc2Vec TaggedDocument for each video's comments;
# append each of those Doc2Vec TaggedDocuments to a "sentences" list;
# create: model = models.Doc2Vec(alpha=.025, min_alpha=.025, min_count=1);
# call: model.build_vocab(sentences);
# train model: for epoch in range(int):... model.train, model.alpha, model.min_alpha

In [31]:
sentences = []
sentences_ordered_dict = OrderedDict()
sent_count = 0
raw_comments = list(comments_df.comments.values)  # this is all of the comments for all videos
for comment_group in raw_comments:
    sent_key = 'sentence' + str(sent_count)
    sentences_ordered_dict[sent_key] = doc2vec.TaggedDocument(
        words=comment_group, tags=["SENT_" + str(sent_count)])
    sentences.append(sentences_ordered_dict[sent_key])
    sent_count += 1
print('TaggedDocument count in sentences is:', len(sentences), '\n')
print(sentences[0])

TaggedDocument count in sentences is: 93 

TaggedDocument(that is some cool grafiti song ,  sick !  ,  damn this is dope ,  i miss the 90s ,  nice beat and cuts man ,  unfortunately they faded so early .  ,  very nice beat bra ,  can you feel it can you feel it can you feeeeel itttt ,  this is super golden age hip hop ,  but still so atmospheric with that string sample !  !  i just dropped a new vid on my channel would love to hear the opinion of a true artist like you over there !  love this beat tho bro !  ,  gj /goodjob !  ,  se puede sacar una super improvisacion buen beat ,    hello .  i want to use ur music for a dance project but i cant find you on facebook ,  and also not on soundcloud .  how can i get your music ?  thank you   ,    whos down to start a rap group   ,  you bouta get the cred for some slick shit bro this beat is fucking absurd ,    its ok if not   ,  can i use this brother ?  ,  hola man queria saber si podia utilizar este  beat para un tema que tengo pensado  ( 

In [ ]:
# Create doc2vec model and build its vocabulary
model = Doc2Vec(alpha=.025, min_alpha=.025, min_count=2)
model.build_vocab(sentences)

In [ ]:
# Get more info about the model
print(model)

In [ ]:
# Find out which objects related to vocabulary are available in the Doc2Vec namespace
[x for x in dir(model) if 'vocab' in x]

In [ ]:
# Look closer at 'vocabulary'
help(model.vocabulary)

In [ ]:
# infer new tokens on new comments documents